# Transfer Learning Tutorial
- [Pytorch Tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) 을 실습해보는 노트북입니다.
- Transfer Learning의 개념을 Pytorch 를 활용하여 적용해보기

- Finetuning the convnet: Convnet 에서의 각 layer 의 weight 값들을 random initialize 하는 것이 아니라, 미리 학습 된 network의 weight 값으로 초기화.
- ConvNet as fixed feature extractor: Final Fully Connected Layer 외에는 나머지 weight 들은 update 를 멈춘다. This last fully connected layer is replaced with a new one with random weights and only this layer is trained

In [1]:
from __future__ import print_function, division

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

%matplotlib inline
plt.ion()

In [4]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip -P ./data

--2019-06-25 16:54:41--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 52.85.192.2, 52.85.192.226, 52.85.192.248, ...
Connecting to download.pytorch.org (download.pytorch.org)|52.85.192.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: './data/hymenoptera_data.zip'

hymenoptera_data.zi 100%[===================>]  45.10M   775KB/s    in 49s     

2019-06-25 16:55:30 (945 KB/s) - './data/hymenoptera_data.zip' saved [47286322/47286322]



In [10]:
!tar -xf ./data/hymenoptera_data.zip -C ./data

## 데이터 로드

In [9]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './data/hymenoptera_data'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Visualize a few images

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
    
inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## Training the model
- Scheduling the learning rate
- Saving the best model

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('EPOCH {}/{}'.format(epoch, num_epochs-1))
        print('-'*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outpus, lables)
                    if phase =='train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f}, ACC: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val ACC: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model    

## Finetuning the Convnet
- 미리 학습된 모델 Load & Reset final fully connected layer

In [11]:
model_ft = models.resnet18(pretrained=True)

In [12]:
num_ftrs = model_ft.fc.in_features
num_ftrs

512

In [13]:
model_ft.fc = nn.Linear(num_ftrs, 2)

In [14]:
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Train and Evaluate

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

## ConvNet as fixed feature extractor